In [1]:
import numpy as np
import os
#import cv2
import shutil
import random as rn
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2022-10-10 21:02:48.017430: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-10 21:02:48.017463: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_dir ="Flowers-Dataset/flowers"

In [3]:
print(os.listdir(data_dir))

['dandelion', 'sunflower', 'tulip', 'rose', 'daisy']


In [4]:
batch_size = 22
img_height = 180
img_width = 180
num_classes = 5

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
test_datagen=ImageDataGenerator(rescale=1./255)


In [6]:
x_train=train_datagen.flow_from_directory(data_dir,target_size=(64,64),class_mode='categorical',batch_size=24)


Found 4317 images belonging to 5 classes.


In [7]:
x_test=test_datagen.flow_from_directory(data_dir,target_size=(64,64),class_mode='categorical',batch_size=24)


Found 4317 images belonging to 5 classes.


In [8]:
x_train.class_indices


{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}

In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4317 files belonging to 5 classes.
Using 3454 files for training.


2022-10-10 21:02:50.096933: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-10 21:02:50.096955: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (shriram-ASUS-TUF-LINUX): /proc/driver/nvidia/version does not exist
2022-10-10 21:02:50.098146: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4317 files belonging to 5 classes.
Using 863 files for validation.


In [11]:
class_names = train_ds.class_names
class_names

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [13]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [14]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.3),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dense(num_classes)
])

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
epochs=50
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/50
157/157 [==============================] - 49s 305ms/step - loss: 1.2778 - accuracy: 0.4441 - val_loss: 1.2945 - val_accuracy: 0.4936
Epoch 2/50
157/157 [==============================] - 48s 303ms/step - loss: 1.0307 - accuracy: 0.5843 - val_loss: 0.9324 - val_accuracy: 0.6130
Epoch 3/50
157/157 [==============================] - 47s 301ms/step - loss: 0.9500 - accuracy: 0.6213 - val_loss: 0.9565 - val_accuracy: 0.6072
Epoch 4/50
157/157 [==============================] - 47s 300ms/step - loss: 0.9062 - accuracy: 0.6465 - val_loss: 0.8913 - val_accuracy: 0.6431
Epoch 5/50
157/157 [==============================] - 47s 301ms/step - loss: 0.8215 - accuracy: 0.6726 - val_loss: 0.8471 - val_accuracy: 0.6790
Epoch 6/50
157/157 [==============================] - 47s 301ms/step - loss: 0.8042 - accuracy: 0.6896 - val_loss: 0.7350 - val_accuracy: 0.7045
Epoch 7/50
157/157 [==============================] - 48s 303ms/step - loss: 0.7729 - accuracy: 0.6989 - val_loss: 0.8984 - val_ac

In [17]:
model.save("Flowers.h5")

In [19]:
from tensorflow.keras.models import Sequential, Model, load_model

model=load_model(r'Flowers.h5')

In [25]:
from tensorflow.keras.preprocessing import image

test_img=image.load_img(r"Flowers-Dataset/flowers/sunflower/44079668_34dfee3da1_n.jpg",target_size=(180,180))

In [26]:
pixels=image.img_to_array(test_img)
pixels=np.expand_dims(pixels,axis=0)

In [27]:
pred=model.predict(pixels)


In [29]:
categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

categories[np.argmax(pred)]


'sunflower'